<a href="https://colab.research.google.com/github/TejasJoshi2005/Modal-Analysis-of-Fluid-Flows/blob/main/POD_and_DMD_1_on_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh
import h5py, os

In [ ]:
import numpy as np
import h5py

# file name
data_set = "LSP_r1300_p40_small.h5"

# open the h5 file
with h5py.File(data_set, "r") as f:
    # Check what datasets are inside
    print("Keys in file:", list(f.keys()))   # should show ['r1300_p40']

    # Load the actual dataset (replace 'U' with your real key)
    U = f['r1300_p40'][:]   # load as numpy array

# define grid sizes
nx = 768  # number of grid points in x direction
ny = 192  # number of grid points in y direction

# discretize the domain
x = np.linspace(0, 32, nx)
y = np.linspace(0, 8, ny)

# check shape
print("U shape:", U.shape)

# reshape if needed
if U.shape[0] == nx*ny:
    n_points, n_time = U.shape
elif U.shape[1] == nx*ny:
    n_time, n_points = U.shape
    U = U.T   # transpose so it matches (n_points, n_time)
else:
    raise ValueError(f"Unexpected dataset shape {U.shape}, doesn't match nx*ny={nx*ny}")

print("Final U shape:", U.shape, "n_points = nx*ny =", nx*ny)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Visualizing two of the snapshots of the vorticity field

#One time step (dt) = 0.125s
plt.figure(figsize=(10,4))
plt.subplot(2,2,1)
plt.pcolormesh(x,y,np.reshape(U[:,10], (ny,nx)), cmap="RdBu_r", vmin=-1, vmax=1)
plt.title("Snapshot of vorticity at 10th time step")
plt.subplot(2,2,2)
plt.pcolormesh(x,y,np.reshape(U[:,200], (ny,nx)), cmap="RdBu_r", vmin=-1, vmax=1)
plt.title("Snapshot at 200th time step")
plt.subplot(2,2,3)
plt.pcolormesh(x,y, np.reshape(U[:,400], (ny,nx)), cmap ="RdBu_r", vmin=-1, vmax=1)
plt.title("Snapshot at 400th time step")
plt.subplot(2,2,4)
plt.pcolormesh(x,y, np.reshape(U[:,999], (ny,nx)), cmap="RdBu_r", vmin=-1, vmax=1)
plt.title("Snapshot at 900th time step")

plt.subplots_adjust(hspace=0.5)
plt.show()

In [ ]:
X = U.astype(np.float32)

# 1) compute temporal mean and center data
mean_field = np.mean(X, axis=1, keepdims=True)
Xc = X - mean_field                               # centered fluctuations

# 2) compute small correlation matrix C = Xc^T Xc  (shape n_time x n_time)
print("Forming correlation matrix C (size: {}x{}) ...".format(n_time, n_time))
C = (Xc.T @ Xc)

# 3) eigen-decompose C (method-of-snapshots).
eigvals, eigvecs = eigh(C)
# reorder descending
idx = eigvals.argsort()[::-1]
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

# 4) singular values and modal energies
tol = 1e-12
eigvals_clipped = np.clip(eigvals, a_min=0, a_max=None)
singular_vals = np.sqrt(eigvals_clipped)
total_energy = np.sum(singular_vals**2)
energies = (singular_vals**2) / total_energy
cum_energy = np.cumsum(energies)

# 5) compute spatial POD modes U_pod = Xc @ V @ Sigma^{-1}

nonzero = singular_vals > tol # avoid division by zero for tiny singular values
r_max = np.sum(nonzero)
print(f"Non-zero POD modes available: {r_max}")
S_inv = np.zeros_like(singular_vals)
S_inv[nonzero] = 1.0 / singular_vals[nonzero]
# modes shape: (n_points, n_time) but only first r_max are meaningful
modes = (Xc @ eigvecs) * S_inv[np.newaxis, :]   # broadcasting; columns are POD modes
# keep only r_max columns
modes = modes[:, :r_max]
singular_vals = singular_vals[:r_max]
energies = energies[:r_max]
cum_energy = cum_energy[:r_max]

# 6) modal time coefficients (amplitudes): a = U^T Xc  => shape (r_max, n_time)
coeffs = modes.T @ Xc



In [ ]:
os.makedirs("pod_outputs", exist_ok=True)

# Plot 1: singular values (scree) and energy decay
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.semilogy(np.arange(1, r_max+1), singular_vals, 'o-')
plt.xlabel('Mode index')
plt.ylabel('Singular value (log)')
plt.title('POD singular values (scree)')

plt.subplot(1,2,2)
plt.plot(np.arange(1, r_max+1), cum_energy, 'o-')
plt.xlabel('Number of modes r')
plt.ylabel('Cumulative energy fraction')
plt.grid(True)
plt.title('Cumulative energy')
plt.axhline(0.9, color='gray', linestyle='--', label='90% energy')
plt.axhline(0.99, color='gray', linestyle=':', label='99% energy')
plt.legend()
plt.tight_layout()
plt.savefig("pod_outputs/scree_and_cumulative_energy.png", dpi=200)
plt.show()

# Suggest r by energy thresholds
r90 = np.searchsorted(cum_energy, 0.90) + 1
r95 = np.searchsorted(cum_energy, 0.95) + 1
r99 = np.searchsorted(cum_energy, 0.99) + 1
print(f"Modes for 90% energy: r90={r90}, 95%: r95={r95}, 99%: r99={r99}")


In [ ]:
# Plot 2: first 6 spatial POD modes (reshape to (ny,nx))
n_show_modes = 6
vmax_mode = np.max(np.abs(modes[:, :n_show_modes]))
plt.figure(figsize=(12,6))
for i in range(n_show_modes):
    ax = plt.subplot(2, 3, i+1)
    mode2d = modes[:, i].reshape((ny, nx))
    pcm = ax.pcolormesh(x, y, mode2d, shading='auto', cmap='RdBu_r',
                        vmin=-vmax_mode, vmax=vmax_mode)
    ax.set_title(f"POD mode {i+1} (energy={energies[i]:.3f})")
    ax.set_xlabel('x'); ax.set_ylabel('y')
    plt.colorbar(pcm, ax=ax, fraction=0.046)
plt.tight_layout()
plt.savefig("pod_outputs/first_6_pod_modes.png", dpi=200)
plt.show()

In [ ]:
# Plot 3: time coefficients of first 4 modes
dt = 0.125
plt.figure(figsize=(10,6))
t = np.arange(n_time) * dt
for i in range(4):
    plt.plot(t, coeffs[i, :].real, label=f"mode {i+1}")
plt.xlabel('time (s)')
plt.ylabel('modal amplitude')
plt.title('Time coefficients (first 4 POD modes)')
plt.legend()
plt.grid(True)
plt.savefig("pod_outputs/coeffs_first4.png", dpi=200)
plt.show()

In [ ]:
# Dynamic Mode Decomposition (DMD)

# 1) Create snapshot matrices X1 and X2 from the original data X
X1 = X[:, :-1]
X2 = X[:, 1:]
print("X1 shape:", X1.shape)
print("X2 shape:", X2.shape)

# 2) SVD of X1
print("Performing SVD on X1...")
U_svd, s_svd, Vt_svd = np.linalg.svd(X1, full_matrices=False)

# Plot singular values from this SVD to help decide the truncation rank
plt.figure(figsize=(6,4))
plt.semilogy(s_svd, 'o-')
plt.title("Singular Values from SVD of X1")
plt.xlabel("Mode index")
plt.ylabel("Singular value (log)")
plt.grid(True)
plt.show()

# 3) Truncate SVD. The plot shows the singular values drop steadily.
# A higher rank captures more detailed dynamics but may also include noise.
# We'll choose r_dmd=50 for a good balance.
r_dmd = 50
print(f"Truncating to rank r_dmd = {r_dmd}")

Ur = U_svd[:, :r_dmd]
Sr = np.diag(s_svd[:r_dmd])
Vr = Vt_svd[:r_dmd, :].T # V is the transpose of Vt

# 4) Compute the low-rank operator Atilde
# Atilde = U_r^T * X2 * V_r * Sigma_r^{-1}
Atilde = Ur.T @ X2 @ Vr @ np.linalg.inv(Sr)

# 5) Eigendecomposition of Atilde to get DMD eigenvalues and low-rank eigenvectors
eigvals_dmd, eigvecs_dmd_low_rank = np.linalg.eig(Atilde)

# 6) Compute full-dimensional DMD modes by projecting back
modes_dmd = Ur @ eigvecs_dmd_low_rank

# 7) Compute continuous-time eigenvalues (growth/decay rates and frequencies)
omega = np.log(eigvals_dmd) / dt # dt=0.125s was defined in a previous cell

# 8) Compute initial amplitudes of each mode
x1 = X[:, 0]
b_amps = np.linalg.pinv(modes_dmd) @ x1

# 9) Sort modes by the magnitude of their initial amplitudes for visualization
amp_sort_indices = np.argsort(np.abs(b_amps))[::-1]
eigvals_dmd_sorted = eigvals_dmd[amp_sort_indices]
modes_dmd_sorted = modes_dmd[:, amp_sort_indices]
omega_sorted = omega[amp_sort_indices]
b_amps_sorted = b_amps[amp_sort_indices]

os.makedirs("dmd_outputs", exist_ok=True)
print("DMD computation complete.")

In [ ]:
# --- DMD Visualization 1: Eigenvalue Spectrum ---

plt.figure(figsize=(12, 5))

# Plot 1: Eigenvalues on the complex plane
ax1 = plt.subplot(1, 2, 1)
unit_circle = plt.Circle((0,0), 1, color='gray', fill=False, linestyle='--')
ax1.add_artist(unit_circle)
sc = ax1.scatter(eigvals_dmd.real, eigvals_dmd.imag, c=np.abs(b_amps), cmap='jet', s=50)
plt.colorbar(sc, label='Initial Amplitude |b|')
ax1.set_xlabel('Re($\\lambda$)')
ax1.set_ylabel('Im($\\lambda$)')
ax1.set_title('DMD Eigenvalues in Complex Plane')
ax1.axis('equal')
Untitled6.ipynb
Untitled6.ipynb_

[ ]
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh
import h5py, os

[ ]
import urllib.request


url = "https://www.dropbox.com/s/nmb9yqxyoswqgkz/cyldata1k.csv?dl=1"
data_set = "cyldata1k.csv"


urllib.request.urlretrieve(url, data_set)


U = np.loadtxt(data_set, delimiter=",")

"""
The dataset contains 1000 snapshots of the vorticity
of the flow past a cylinder at Reynolds number, Re = 200.

Vorticity: Rotation of the fluid at a particular point.
Mathematically its the curl of the velocity field

"""

# Grid details
nx = 768  # number of grid points in x direction
ny = 192  # number of grid points in y direction

"""
A grid point is a discrete location in space where the cfd simulation stores a value.
Here that value is the value of vorticity

"""

x = np.linspace(0, 32, nx)  # discretizing in x direction
y = np.linspace(0, 8, ny)   # discretizing in y direction

# Check shape
n_points, n_time = U.shape
print("U shape:", U.shape, "n_points = nx*ny=", nx*ny)
assert n_points == nx * ny


[ ]
#Visualizing two of the snapshots of the vorticity field

#One time step (dt) = 0.125s
plt.figure(figsize=(10,4))
plt.subplot(2,2,1)
plt.pcolormesh(x,y,np.reshape(U[:,10], (ny,nx)), cmap="RdBu_r", vmin=-1, vmax=1)
plt.title("Snapshot of vorticity at 10th time step")
plt.subplot(2,2,2)
plt.pcolormesh(x,y,np.reshape(U[:,200], (ny,nx)), cmap="RdBu_r", vmin=-1, vmax=1)
plt.title("Snapshot at 200th time step")
plt.subplot(2,2,3)
plt.pcolormesh(x,y, np.reshape(U[:,400], (ny,nx)), cmap ="RdBu_r", vmin=-1, vmax=1)
plt.title("Snapshot at 400th time step")
plt.subplot(2,2,4)
plt.pcolormesh(x,y, np.reshape(U[:,999], (ny,nx)), cmap="RdBu_r", vmin=-1, vmax=1)
plt.title("Snapshot at 900th time step")

plt.subplots_adjust(hspace=0.5)
plt.show()

[ ]
X = U.astype(np.float32)

# 1) compute temporal mean and center data
mean_field = np.mean(X, axis=1, keepdims=True)
Xc = X - mean_field                               # centered fluctuations

# 2) compute small correlation matrix C = Xc^T Xc  (shape n_time x n_time)
print("Forming correlation matrix C (size: {}x{}) ...".format(n_time, n_time))
C = (Xc.T @ Xc)

# 3) eigen-decompose C (method-of-snapshots).
eigvals, eigvecs = eigh(C)
# reorder descending
idx = eigvals.argsort()[::-1]
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

# 4) singular values and modal energies
tol = 1e-12
eigvals_clipped = np.clip(eigvals, a_min=0, a_max=None)
singular_vals = np.sqrt(eigvals_clipped)
total_energy = np.sum(singular_vals**2)
energies = (singular_vals**2) / total_energy
cum_energy = np.cumsum(energies)

# 5) compute spatial POD modes U_pod = Xc @ V @ Sigma^{-1}

nonzero = singular_vals > tol # avoid division by zero for tiny singular values
r_max = np.sum(nonzero)
print(f"Non-zero POD modes available: {r_max}")
S_inv = np.zeros_like(singular_vals)
S_inv[nonzero] = 1.0 / singular_vals[nonzero]
# modes shape: (n_points, n_time) but only first r_max are meaningful
modes = (Xc @ eigvecs) * S_inv[np.newaxis, :]   # broadcasting; columns are POD modes
# keep only r_max columns
modes = modes[:, :r_max]
singular_vals = singular_vals[:r_max]
energies = energies[:r_max]
cum_energy = cum_energy[:r_max]

# 6) modal time coefficients (amplitudes): a = U^T Xc  => shape (r_max, n_time)
coeffs = modes.T @ Xc



[ ]
os.makedirs("pod_outputs", exist_ok=True)

# Plot 1: singular values (scree) and energy decay
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.semilogy(np.arange(1, r_max+1), singular_vals, 'o-')
plt.xlabel('Mode index')
plt.ylabel('Singular value (log)')
plt.title('POD singular values (scree)')

plt.subplot(1,2,2)
plt.plot(np.arange(1, r_max+1), cum_energy, 'o-')
plt.xlabel('Number of modes r')
plt.ylabel('Cumulative energy fraction')
plt.grid(True)
plt.title('Cumulative energy')
plt.axhline(0.9, color='gray', linestyle='--', label='90% energy')
plt.axhline(0.99, color='gray', linestyle=':', label='99% energy')
plt.legend()
plt.tight_layout()
plt.savefig("pod_outputs/scree_and_cumulative_energy.png", dpi=200)
plt.show()

# Suggest r by energy thresholds
r90 = np.searchsorted(cum_energy, 0.90) + 1
r95 = np.searchsorted(cum_energy, 0.95) + 1
r99 = np.searchsorted(cum_energy, 0.99) + 1
print(f"Modes for 90% energy: r90={r90}, 95%: r95={r95}, 99%: r99={r99}")


[ ]
# Plot 2: first 6 spatial POD modes (reshape to (ny,nx))
n_show_modes = 6
vmax_mode = np.max(np.abs(modes[:, :n_show_modes]))
plt.figure(figsize=(12,6))
for i in range(n_show_modes):
    ax = plt.subplot(2, 3, i+1)
    mode2d = modes[:, i].reshape((ny, nx))
    pcm = ax.pcolormesh(x, y, mode2d, shading='auto', cmap='RdBu_r',
                        vmin=-vmax_mode, vmax=vmax_mode)
    ax.set_title(f"POD mode {i+1} (energy={energies[i]:.3f})")
    ax.set_xlabel('x'); ax.set_ylabel('y')
    plt.colorbar(pcm, ax=ax, fraction=0.046)
plt.tight_layout()
plt.savefig("pod_outputs/first_6_pod_modes.png", dpi=200)
plt.show()

[ ]
# Plot 3: time coefficients of first 4 modes
dt = 0.125
plt.figure(figsize=(10,6))
t = np.arange(n_time) * dt
for i in range(4):
    plt.plot(t, coeffs[i, :].real, label=f"mode {i+1}")
plt.xlabel('time (s)')
plt.ylabel('modal amplitude')
plt.title('Time coefficients (first 4 POD modes)')
plt.legend()
plt.grid(True)
plt.savefig("pod_outputs/coeffs_first4.png", dpi=200)
plt.show()

[ ]
# Dynamic Mode Decomposition (DMD)

# 1) Create snapshot matrices X1 and X2 from the original data X
X1 = X[:, :-1]
X2 = X[:, 1:]
print("X1 shape:", X1.shape)
print("X2 shape:", X2.shape)

# 2) SVD of X1
print("Performing SVD on X1...")
U_svd, s_svd, Vt_svd = np.linalg.svd(X1, full_matrices=False)

# Plot singular values from this SVD to help decide the truncation rank
plt.figure(figsize=(6,4))
plt.semilogy(s_svd, 'o-')
plt.title("Singular Values from SVD of X1")
plt.xlabel("Mode index")
plt.ylabel("Singular value (log)")
plt.grid(True)
plt.show()

# 3) Truncate SVD. The plot shows the singular values drop steadily.
# A higher rank captures more detailed dynamics but may also include noise.
# We'll choose r_dmd=50 for a good balance.
r_dmd = 50
print(f"Truncating to rank r_dmd = {r_dmd}")

Ur = U_svd[:, :r_dmd]
Sr = np.diag(s_svd[:r_dmd])
Vr = Vt_svd[:r_dmd, :].T # V is the transpose of Vt

# 4) Compute the low-rank operator Atilde
# Atilde = U_r^T * X2 * V_r * Sigma_r^{-1}
Atilde = Ur.T @ X2 @ Vr @ np.linalg.inv(Sr)

# 5) Eigendecomposition of Atilde to get DMD eigenvalues and low-rank eigenvectors
eigvals_dmd, eigvecs_dmd_low_rank = np.linalg.eig(Atilde)

# 6) Compute full-dimensional DMD modes by projecting back
modes_dmd = Ur @ eigvecs_dmd_low_rank

# 7) Compute continuous-time eigenvalues (growth/decay rates and frequencies)
omega = np.log(eigvals_dmd) / dt # dt=0.125s was defined in a previous cell

# 8) Compute initial amplitudes of each mode
x1 = X[:, 0]
b_amps = np.linalg.pinv(modes_dmd) @ x1

# 9) Sort modes by the magnitude of their initial amplitudes for visualization
amp_sort_indices = np.argsort(np.abs(b_amps))[::-1]
eigvals_dmd_sorted = eigvals_dmd[amp_sort_indices]
modes_dmd_sorted = modes_dmd[:, amp_sort_indices]
omega_sorted = omega[amp_sort_indices]
b_amps_sorted = b_amps[amp_sort_indices]

os.makedirs("dmd_outputs", exist_ok=True)
print("DMD computation complete.")

[ ]

ax1.grid(True)

# Plot 2: Growth/Decay vs. Frequency
ax2 = plt.subplot(1, 2, 2)
# Frequencies are in rad/s, convert to Hz (freq = omega_imag / (2*pi))
frequencies = omega.imag / (2 * np.pi)
growth_rates = omega.real
sc2 = ax2.scatter(frequencies, growth_rates, c=np.abs(b_amps), cmap='jet', s=50)
plt.colorbar(sc2, label='Initial Amplitude |b|')
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('Growth/Decay Rate (1/s)')
ax2.set_title('DMD Spectrum: Growth vs. Frequency')
ax2.grid(True)
ax2.axhline(0, color='gray', linestyle='--') # Zero growth line

plt.tight_layout()
plt.savefig("dmd_outputs/dmd_spectrum.png", dpi=200)
plt.show()

# Identify the main frequency from the most energetic mode
# Filter for positive frequencies to find the dominant oscillation
positive_freq_indices = np.where(frequencies > 0.01) # Ignore near-zero frequencies
dominant_freq_idx = positive_freq_indices[0][np.argmax(np.abs(b_amps[positive_freq_indices]))]
dominant_freq = frequencies[dominant_freq_idx]
print(f"The most energetic oscillating frequency is approximately {dominant_freq:.4f} Hz.")

In [ ]:
# --- DMD Visualization 2: Spatial DMD Modes ---
# We visualize the real part of the sorted modes

n_show_modes = 6
# Normalize color scale based on the max value in the modes being shown
vmax_dmd_mode = np.max(np.abs(modes_dmd_sorted[:, :n_show_modes].real))

plt.figure(figsize=(12, 6))
for i in range(n_show_modes):
    ax = plt.subplot(2, 3, i + 1)
    mode_dmd_2d = modes_dmd_sorted[:, i].real.reshape((ny, nx))
    freq = omega_sorted[i].imag / (2 * np.pi)
    growth = omega_sorted[i].real

    pcm = ax.pcolormesh(x, y, mode_dmd_2d, shading='auto', cmap='RdBu_r',
                        vmin=-vmax_dmd_mode, vmax=vmax_dmd_mode)

    title_str = (f"DMD Mode {i+1}\\n"
                 f"f={freq:.3f} Hz, g={growth:.3f}")
    ax.set_title(title_str)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.colorbar(pcm, ax=ax, fraction=0.046)

plt.tight_layout()
plt.savefig("dmd_outputs/first_6_dmd_modes.png", dpi=200)
plt.show()

In [ ]:
# --- DMD Visualization 3: Reconstruction ---

# Reconstruct the data using the DMD modes, eigenvalues, and amplitudes
time_dynamics = np.zeros((r_dmd, n_time), dtype=complex)
t_vec = np.arange(n_time) * dt
for i in range(n_time):
    # Use original unsorted eigenvalues and amplitudes for reconstruction
    time_dynamics[:, i] = b_amps * np.exp(omega * t_vec[i])

X_dmd_reconstructed = (modes_dmd @ time_dynamics).real

# Compare an original snapshot with the DMD reconstruction
snap_idx = 400
original_snap = X[:, snap_idx].reshape(ny, nx)
reconstructed_snap = X_dmd_reconstructed[:, snap_idx].reshape(ny, nx)
error_snap = original_snap - reconstructed_snap

# Calculate relative L2 error for the snapshot
rel_error = np.linalg.norm(error_snap) / np.linalg.norm(original_snap)
print(f"Relative L2 error for snapshot {snap_idx}: {rel_error:.4f}")

plt.figure(figsize=(15, 4))
vmax_snap = np.max(np.abs(original_snap))

plt.subplot(1, 3, 1)
plt.pcolormesh(x, y, original_snap, cmap="RdBu_r", vmin=-vmax_snap, vmax=vmax_snap)
plt.title(f"Original Snapshot at t={snap_idx*dt}s")
plt.xlabel('x'); plt.ylabel('y')

plt.subplot(1, 3, 2)
plt.pcolormesh(x, y, reconstructed_snap, cmap="RdBu_r", vmin=-vmax_snap, vmax=vmax_snap)
plt.title(f"DMD Reconstruction (r={r_dmd})")
plt.xlabel('x'); plt.ylabel('y')

plt.subplot(1, 3, 3)
vmax_err = np.max(np.abs(error_snap))
plt.pcolormesh(x, y, error_snap, cmap="bwr", vmin=-vmax_err, vmax=vmax_err)
plt.title(f"Absolute Error (Max Err: {vmax_err:.2f})")
plt.xlabel('x'); plt.ylabel('y')

plt.tight_layout()
plt.savefig("dmd_outputs/dmd_reconstruction.png", dpi=200)
plt.show()